In [1]:
import pickle
import numpy as np
import pandas as pd

from tensorflow.keras.models import load_model
from gensim.models import Word2Vec
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [2]:
REVIEWS_DATASET_DIR = '../../dataset'
tourism_reviews_en = pd.read_csv(f"{REVIEWS_DATASET_DIR}/eng_reviews_emotion_classify.csv", encoding='utf-8')

columns_to_use = ['helpful_votes', 'location_id', 'review_id', 'review',
                  'review_subject', 'trip_type', 'rating',
                  'location_name', 'province', 'place_id', 'emotion', 'cleaned_review']
columns_to_train = ['location_id', 'review', 'rating']
review_df = tourism_reviews_en[columns_to_use]

In [3]:
review_df.head(1)

,helpful_votes,location_id,review_id,review,review_subject,trip_type,rating,location_name,province,place_id,emotion,cleaned_review
0,0,2209612,953596359,Besides elegant grand palace and wat pra kaew ...,"The place is nearby grand palace, in the area ...",Family,5,Wat Ratchabophit,Bangkok,P03013220,joy,besides elegant grand palace <OOV> wat pra kae...


In [4]:
import sys
import os

# Define the path to the parent directory where `utils` is located
parent_directory = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))

# Add the parent directory to the Python path
sys.path.append(parent_directory)

# Verify the updated path
print("Updated sys.path:", sys.path)

Updated sys.path: ['/opt/homebrew/Cellar/python@3.12/3.12.5/Frameworks/Python.framework/Versions/3.12/lib/python312.zip', '/opt/homebrew/Cellar/python@3.12/3.12.5/Frameworks/Python.framework/Versions/3.12/lib/python3.12', '/opt/homebrew/Cellar/python@3.12/3.12.5/Frameworks/Python.framework/Versions/3.12/lib/python3.12/lib-dynload', '', '/Users/ittichaiboonyarakthunya/Library/Caches/pypoetry/virtualenvs/is-project-thailand-tourism-recommendation-qwL-LsgH-py3.12/lib/python3.12/site-packages', '/Users/ittichaiboonyarakthunya/Library/Caches/pypoetry/virtualenvs/is-project-thailand-tourism-recommendation-qwL-LsgH-py3.12/lib/python3.12/site-packages/setuptools/_vendor', '/Users/ittichaiboonyarakthunya/Documents/WorkDir/developments/IS_Project_Thailand_Tourism_Recommendation']


In [5]:
import sys
import os

# Set the path to the parent directory relative to where the notebook is located
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../../utils')))

In [6]:
from utils.data_preparation import DataPreparation

DP = DataPreparation()
DP.check_nltk_path()


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /Users/ittichaiboonyarakth
[nltk_data]    |     unya/Documents/WorkDir/developments/IS_Project_Th
[nltk_data]    |     ailand_Tourism_Recommendation/pretained_or_finetu
[nltk_data]    |     ne-models/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /Users/ittichaiboonyara
[nltk_data]    |     kthunya/Documents/WorkDir/developments/IS_Project
[nltk_data]    |     _Thailand_Tourism_Recommendation/pretained_or_fin
[nltk_data]    |     etune-models/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to /Us
[nltk_data]    |     ers/ittichaiboonyarakthunya/Documents/WorkDir/dev
[nltk_data]    |     elopments/IS_Project_Thailand_Tourism_Recommendat
[nltk_data]    |     ion/pretained_or_finetune-models/nltk_data...
[nltk_data]    |   Package averag

Numpy version: 1.26.4
Pandas version: 2.2.0
True


[nltk_data]    |   Package wordnet31 is already up-to-date!
[nltk_data]    | Downloading package wordnet_ic to /Users/ittichaiboon
[nltk_data]    |     yarakthunya/Documents/WorkDir/developments/IS_Pro
[nltk_data]    |     ject_Thailand_Tourism_Recommendation/pretained_or
[nltk_data]    |     _finetune-models/nltk_data...
[nltk_data]    |   Package wordnet_ic is already up-to-date!
[nltk_data]    | Downloading package words to /Users/ittichaiboonyarak
[nltk_data]    |     thunya/Documents/WorkDir/developments/IS_Project_
[nltk_data]    |     Thailand_Tourism_Recommendation/pretained_or_fine
[nltk_data]    |     tune-models/nltk_data...
[nltk_data]    |   Package words is already up-to-date!
[nltk_data]    | Downloading package ycoe to /Users/ittichaiboonyarakt
[nltk_data]    |     hunya/Documents/WorkDir/developments/IS_Project_T
[nltk_data]    |     hailand_Tourism_Recommendation/pretained_or_finet
[nltk_data]    |     une-models/nltk_data...
[nltk_data]    |   Package ycoe is already

In [7]:

review_df['cleaned_review'] = review_df['review'].apply(DP.clean_text)

sentences = [row.split() for row in review_df['cleaned_review']]
word2vec_model = Word2Vec(sentences, vector_size=300, window=5, min_count=1, workers=4)  # เปลี่ยน vector_size เป็น 300

/var/folders/fk/449tdf_12hzd2r7ps4659q0m0000gn/T/ipykernel_52309/3405331118.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  review_df['cleaned_review'] = review_df['review'].apply(DP.clean_text)


In [ ]:
# Parameters
embedding_dim = 300  # Assuming Word2Vec has 300 dimensions
gru_units = 128
dropout_rate = 0.5
num_classes = 1  # Binary classification
max_sequence_length = 100  # Maximum length of input sequences
ngram_range = (1, 2)  # Unigrams and bigrams
max_features = 20000  

# Create the embedding matrix from the Word2Vec model
word_index = {word: i for i, word in enumerate(word2vec_model.wv.index_to_key)}
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))

In [ ]:
review_df['embedding'] = review_df['cleaned_review'].apply(lambda x: DP.text_to_word2vec(x, word2vec_model))


In [8]:
def load_svn_model(model_path):
    with open(model_path, 'rb') as file:
        return pickle.load(file)

def load_keras_model(model_path):
    return load_model(model_path)


: 

In [ ]:
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd

# Load your dataframe (example: review_df)
# review_df = pd.read_csv('your_data.csv')  # Modify as needed

# Assuming review_df has features and labels
X = review_df.drop('label', axis=1).values  # Modify 'label' if needed
y = review_df['label'].values  # Modify 'label' if needed

# Define K-Fold
kf = KFold(n_splits=5, shuffle=True, random_state=42)


In [ ]:
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Define a function to evaluate a model
def evaluate_model(model, X_train, y_train, X_val, y_val):
    # Train the model
    model.fit(X_train, y_train, epochs=5, batch_size=32, verbose=1)  # Modify epochs and batch_size as needed
    
    # Predict and evaluate
    y_pred = model.predict(X_val)
    y_pred = np.argmax(y_pred, axis=1)  # Assuming a classification task
    
    # Calculate metrics
    accuracy = accuracy_score(y_val, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_val, y_pred, average='macro')
    
    return accuracy, precision, recall, f1

# Paths to your models
model_paths = {
    "CNN": "CNN_models_sample.keras",
    "GRU": "GRU_model_sample.keras",
    "Hybrid-CNN-LSTM": "Hybrid-CNN-LSTM_models_sample.keras",
    "LSTM": "LSTM_models_sample.keras",
    "SVN": "SVN_model_sample.keras"  # If SVN is a deep learning model, otherwise handle separately
}

results = {model_name: {"accuracy": [], "precision": [], "recall": [], "f1": []} for model_name in model_paths}

# Perform K-Fold Cross-Validation
for train_index, val_index in kf.split(X):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]
    
    for model_name, model_path in model_paths.items():
        model = load_keras_model(model_path)
        accuracy, precision, recall, f1 = evaluate_model(model, X_train, y_train, X_val, y_val)
        
        results[model_name]["accuracy"].append(accuracy)
        results[model_name]["precision"].append(precision)
        results[model_name]["recall"].append(recall)
        results[model_name]["f1"].append(f1)

# Print results
for model_name, metrics in results.items():
    print(f"Model: {model_name}")
    print(f"  Mean Accuracy: {np.mean(metrics['accuracy']):.4f} (+/- {np.std(metrics['accuracy']):.4f})")
    print(f"  Mean Precision: {np.mean(metrics['precision']):.4f} (+/- {np.std(metrics['precision']):.4f})")
    print(f"  Mean Recall: {np.mean(metrics['recall']):.4f} (+/- {np.std(metrics['recall']):.4f})")
    print(f"  Mean F1-Score: {np.mean(metrics['f1']):.4f} (+/- {np.std(metrics['f1']):.4f})")
    print(f"\n")
